# Parametric Study Example

This notebook demonstrates how to run a parametric study varying
insulation thickness and comparing results across variants.

## Setup

First, ensure you have idfkit installed with plotting support:

```bash
pip install idfkit[plot,dataframes]
```

In [ ]:
from idfkit import new_document

## Create a Base Model

We'll create a simple single-zone model with configurable wall insulation.

In [ ]:
def create_model(insulation_thickness: float):
    """Create a simple box model with specified insulation thickness."""
    model = new_document(version=(24, 1, 0))

    # Building and zone
    model.add("Building", "Test Building", north_axis=0, terrain="City")
    model.add("Zone", "Zone1", x_origin=0, y_origin=0, z_origin=0)

    # Materials
    model.add(
        "Material",
        "Concrete",
        roughness="MediumRough",
        thickness=0.2,
        conductivity=1.73,
        density=2243,
        specific_heat=837,
    )
    model.add(
        "Material",
        "Insulation",
        roughness="MediumSmooth",
        thickness=insulation_thickness,
        conductivity=0.04,
        density=32,
        specific_heat=830,
    )

    # Construction
    model.add(
        "Construction",
        "Wall_Construction",
        outside_layer="Concrete",
        layer_2="Insulation",
    )

    # ... add surfaces, etc. (simplified for this example)

    return model


# Create variants
thicknesses = [0.025, 0.05, 0.075, 0.10, 0.15]
models = {t: create_model(t) for t in thicknesses}

print(f"Created {len(models)} model variants")

## Run Batch Simulation

Use `simulate_batch()` to run all variants in parallel.

> **Note:** This requires EnergyPlus to be installed. The code below
> shows the pattern but won't execute without EnergyPlus.

In [ ]:
# Simulation code (requires EnergyPlus)
"""
from idfkit.simulation import simulate_batch, SimulationJob, SimulationCache

# Create simulation jobs
jobs = [
    SimulationJob(
        model=model,
        weather="chicago.epw",
        label=f"insulation-{thickness}m",
        design_day=True,
    )
    for thickness, model in models.items()
]

# Run with progress tracking
cache = SimulationCache()

def progress(completed, total, label, success):
    status = "OK" if success else "FAIL"
    print(f"[{completed}/{total}] {label}: {status}")

batch = simulate_batch(
    jobs,
    max_workers=4,
    cache=cache,
    progress=progress,
)

print(f"\nCompleted: {len(batch.succeeded)}/{len(batch)}")
"""
print("Simulation code shown above (requires EnergyPlus)")

## Analyze Results

Extract and compare results across variants.

In [ ]:
# Analysis code (requires simulation results)
"""
import matplotlib.pyplot as plt

# Extract peak cooling loads
results_data = []

for thickness, result in zip(thicknesses, batch):
    if result.success:
        # Get cooling design day results
        ts = result.sql.get_timeseries(
            "Zone Ideal Loads Supply Air Total Cooling Energy",
            "Zone1",
        )
        peak_cooling = max(ts.values)
        results_data.append({
            "thickness": thickness * 100,  # Convert to cm
            "peak_cooling": peak_cooling / 1000,  # Convert to kW
        })

# Plot results
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(
    [d["thickness"] for d in results_data],
    [d["peak_cooling"] for d in results_data],
)
ax.set_xlabel("Insulation Thickness (cm)")
ax.set_ylabel("Peak Cooling Load (kW)")
ax.set_title("Impact of Insulation Thickness on Peak Cooling Load")

plt.tight_layout()
plt.show()
"""
print("Analysis code shown above (requires simulation results)")

## Summary

This example demonstrated:

1. Creating model variants with `model.copy()` and field modification
2. Running parallel simulations with `simulate_batch()`
3. Using `SimulationCache` to avoid redundant work
4. Extracting and comparing time-series results

## Next Steps

- [Design Day Sizing](sizing-workflow.ipynb) — Apply weather data to models
- [Batch Processing](../simulation/batch.md) — Detailed batch guide
- [SQL Queries](../simulation/sql-queries.md) — Query result data